In [18]:
import pandas as pd;
import re

df = pd.read_excel('../Processadores.xlsx')

In [19]:
df['preco_anterior'] = df['preco_anterior'].str.replace('R\$', '', regex=True)
df['preco_anterior'] = df['preco_anterior'].str.replace('.', '', regex=False) 
df['preco_anterior'] = df['preco_anterior'].str.replace(',', '.', regex=False)
df['preco_anterior'] = df['preco_anterior'].str.strip()
print(df['preco_anterior'])

0      2222.21
1          NaN
2      1222.21
3      1326.22
4      1068.17
        ...   
369        NaN
370        NaN
371        NaN
372        NaN
373        NaN
Name: preco_anterior, Length: 374, dtype: object


<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
C:\Users\48260449846\AppData\Local\Temp\ipykernel_41724\4253348766.py:1: SyntaxWarning: invalid escape sequence '\$'
  df['preco_anterior'] = df['preco_anterior'].str.replace('R\$', '', regex=True)


In [20]:
df['preco_anterior'] = df['preco_anterior'].astype(float)

In [21]:
df['preco_anterior'].fillna('Desconhecido', inplace=True)
print(df['preco_anterior'])

0           2222.21
1      Desconhecido
2           1222.21
3           1326.22
4           1068.17
           ...     
369    Desconhecido
370    Desconhecido
371    Desconhecido
372    Desconhecido
373    Desconhecido
Name: preco_anterior, Length: 374, dtype: object


C:\Users\48260449846\AppData\Local\Temp\ipykernel_41724\350384653.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['preco_anterior'].fillna('Desconhecido', inplace=True)
C:\Users\48260449846\AppData\Local\Temp\ipykernel_41724\350384653.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Desconhecido' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['preco_anterior'].fillna('Desconhecido', inplac

In [22]:
df['preco_atual'] = df['preco_atual'].str.replace('R\$', '', regex=True)
df['preco_atual'] = df['preco_atual'].str.replace('.', '', regex=False) 
df['preco_atual'] = df['preco_atual'].str.replace(',', '.', regex=False)
df['preco_atual'] = df['preco_atual'].str.strip()
print(df['preco_atual'])

0      1749.99
1       656.01
2       999.99
3       599.99
4       749.99
        ...   
369    1664.99
370     399.90
371    3160.23
372     116.04
373    2635.38
Name: preco_atual, Length: 374, dtype: object


<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
C:\Users\48260449846\AppData\Local\Temp\ipykernel_41724\2266943591.py:1: SyntaxWarning: invalid escape sequence '\$'
  df['preco_atual'] = df['preco_atual'].str.replace('R\$', '', regex=True)


In [23]:
df['preco_atual'] = df['preco_atual'].astype(float)
print(df['preco_atual'])

0      1749.99
1       656.01
2       999.99
3       599.99
4       749.99
        ...   
369    1664.99
370     399.90
371    3160.23
372     116.04
373    2635.38
Name: preco_atual, Length: 374, dtype: float64


In [24]:
print(df['preco_atual'].sum())

768644.0800000001


In [25]:
df = df.dropna()

In [26]:
# Extrações do título
MARCAS = {
    'AMD': 'AMD',
    'Intel': 'Intel',
    'Amd': 'Amd'
}

def extrair_info(titulo):
    marca = next((v for k, v in MARCAS.items() if k in titulo), 'Desconhecida')

    modelo_match = re.search(r'(?i)(Ryzen \d+\s*[\w\d\-+]*|Threadripper\s*Pro\s*\d*\w*|Core\s*i\d+[- ]?\d*\w*|Core\s*I\d+[- ]?\w*|Celeron\s*\w+|Pentium\s*\d+\s*Gold\s*\w+|Ultra\s*\d+\w*|Athlon\s*\d+\w*)', titulo)
    modelo = modelo_match.group(0).strip() if modelo_match else 'Desconhecido'

    # Frequência base: sem "Max Turbo" e fora de parênteses (simplificado para evitar erro)
    freq_base = re.search(r'(?i)(\d+[\.,]?\d*)\s*Ghz\b(?![^()]*Max\s*Turbo)', titulo)

    # Frequência turbo: variações com "Max Turbo", dentro ou fora de parênteses
    freq_turbo = re.search(r'(?i)(?:(\d+[\.,]?\d*)\s*Ghz\s*Max\s*Turbo|Max\s*Turbo\s*(\d+[\.,]?\d*)\s*Ghz|Turbo\s*clock\s*(\d+[\.,]?\d*)\s*Ghz)', titulo)

    turbo_val = None
    if freq_turbo:
        for group in freq_turbo.groups():
            if group:
                turbo_val = group
                break

    cache_match = re.search(r'(?i)Cache\s*(\d+)\s*MB', titulo)
    if not cache_match:
        cache_match = re.search(r'(?i)(\d+)\s*MB\s*Cache', titulo)

    def to_float(value):
        try:
            return float(value.replace(',', '.'))
        except:
            return None

    return pd.Series({
        'marca': marca,
        'modelo': modelo,
        'freq_base_GHz': to_float(freq_base.group(1)) if freq_base else None,
        'freq_turbo_GHz': to_float(turbo_val) if turbo_val else None,
        'cache_MB': int(cache_match.group(1)) if cache_match else None
    })

info_extraida = df['titulo'].apply(extrair_info)
df = pd.concat([df, info_extraida], axis=1)



In [27]:
df['marca'] = df['marca'].replace('Amd', 'AMD')

In [28]:
df['em_estoque'] = df['em_estoque'].str.replace('Restam', '')
df['em_estoque'] = df['em_estoque'].str.replace('unid.', '')
df['em_estoque'] = df['em_estoque'].str.strip()
df['em_estoque'] = df['em_estoque'].astype(int)

In [29]:
df['freq_base_GHz'] = df['freq_base_GHz'].fillna(df['freq_base_GHz'].median())
df['freq_turbo_GHz'] = df['freq_turbo_GHz'].fillna(df['freq_turbo_GHz'].median())
df['cache_MB'] = df['cache_MB'].fillna(df['cache_MB'].median())

In [30]:
df.to_excel('processadores_filtrados.xlsx', index=False)

In [31]:
print(df['preco_atual'].sum())
print(df['preco_anterior'].sum())
print(df)

218905.56
302683.94000000006
                                                titulo  preco_atual  \
6    Processador AMD Ryzen 7 5700X, 3.4GHz (4.6GHz ...      1319.99   
11   Processador AMD Ryzen 5 9600X, 3.9 GHz (5.4 GH...      1649.99   
18   Processador Intel Core i5-14600K, 14ª Geração,...      1749.99   
34   Processador Intel Core i7-12700F, 2.1GHz (4.9G...      1699.99   
35   Processador AMD Ryzen 7 8700G, 4.2GHz (5.1GHz ...      1949.99   
..                                                 ...          ...   
226  Processador AMD Ryzen Threadripper Pro 5995WX,...     37999.99   
236  Processador Intel Core I9-12900KF, 3.2GHz, Cac...      2979.00   
249  Processador Intel Pentium Gold G6405, 10ª Gera...       509.00   
289  Processador Intel Core Ultra 7 265kf Arrow Lak...      2669.00   
331  Processador Intel Ultra 5 245k Arrow Lake 4.2g...      2469.00   

    preco_anterior  em_estoque  marca                   modelo  freq_base_GHz  \
6           2840.9          80    AMD